<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/10-Optimizacion-Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Escuela de Ciencias Físicas y Matemática</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optmización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Melani Loor &bull; 2024-02
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

A lo largo de este taller, aplicaremos optimización de hiperparámetros en un modelo que elijas.

Los paquetes necesarios son:

In [ ]:
# Paquetes necesarios
import pandas as pd  # Manejo de datos
import matplotlib.pyplot as plt  # Visualización

from sklearn.model_selection import train_test_split # División de datos
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métrica de evaluación

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold  # Búsqueda de hiperparámetros
from sklearn.ensemble import RandomForestClassifier

import joblib
import pickle

from sklearn.model_selection import RandomizedSearchCV



---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilzar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

In [ ]:
# Leer los datos
data = pd.read_csv('https://raw.githubusercontent.com/mnloor/Aprendizaje-Automatico/refs/heads/main/Datos/diabetes.csv')

data.rename(columns={
    "Pregnancies": "Embarazos",
    "Glucose": "Glucosa",
    "BloodPressure": "Presión_Arterial",
    "SkinThickness": "Grosor_de_Piel",
    "Insulin": "Insulina",
    "BMI": "IMC",
    "DiabetesPedigreeFunction": "Funcion_Hereditaria_Diabetes",
    "Age": "Edad",
    "Outcome": "Resultado"
}, inplace=True)


# Verificar nombres de columnas
data.columns

# Tomo las columnas de interés
numeric_cols = ['Embarazos', 'Glucosa', 'Presión_Arterial', 'Grosor_de_Piel', 'Insulina','IMC','Funcion_Hereditaria_Diabetes','Edad']
class_col = ['Resultado']

# Muestro los primeros registros
display(data.head())

,Embarazos,Glucosa,Presión_Arterial,Grosor_de_Piel,Insulina,IMC,Funcion_Hereditaria_Diabetes,Edad,Resultado
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


**Descripción de datos**

In [ ]:
data.describe()

,Embarazos,Glucosa,Presión_Arterial,Grosor_de_Piel,Insulina,IMC,Funcion_Hereditaria_Diabetes,Edad,Resultado
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
X = data[numeric_cols]
y = data['Resultado']


### <font color='264CC7'> Modelo </font>


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Selecciona el mejor modelo de las clases anteriores.
<ul>
  <li>Muestra los hiperparámetros del modelo.</li>
  <li>Consulta qué significan al menos 4 hiperparámetros.</li>
  <li>Selecciona los hiperparámetros que deseas optimizar, al menos 3.</li>
</ul>
</div>

In [ ]:
# Crear y entrenar un arbol con ganancia de información
modelo_base = RandomForestClassifier(random_state=68)

# Parámetros del modelo
modelo_base.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 68,
 'verbose': 0,
 'warm_start': False}

**Consulta de significados**

**n_estimators:** Número de árboles en el bosque. Más árboles suelen mejorar la estabilidad y precisión del modelo, pero también incrementan el tiempo de cómputo.

**max_depth:** Profundidad máxima de cada árbol. Limitarla ayuda a controlar el sobreajuste.

**max_features:** Cantidad máxima de características consideradas para dividir un nodo. 'sqrt' selecciona la raíz cuadrada del número total de características.

**min_samples_split:** Número mínimo de muestras necesarias para dividir un nodo interno. Evita divisiones excesivas y ayuda a generalizar el modelo.

**Elección de hiperparámetros a optimizar**

Voy a optimizar los hiperparámetros de n_estimators, 'max_depth' y 'criterion'

### <font color='264CC7'> Optimización por GridSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica GridSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 3 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [ ]:
parametros = {'n_estimators': [5, 10,15,20,25,30],
              'max_depth': [None, 3, 5,8,11,14],
              'criterion':['gini','entropy']}
k_fold = KFold(n_splits=5, shuffle=True, random_state=68)

modelo = GridSearchCV(modelo_base, parametros, cv=k_fold, scoring='accuracy')
modelo

GridSearchCV(cv=KFold(n_splits=5, random_state=68, shuffle=True),
             estimator=RandomForestClassifier(random_state=68),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 3, 5, 8, 11, 14],
                         'n_estimators': [5, 10, 15, 20, 25, 30]},
             scoring='accuracy')

In [ ]:
modelo.fit(X, y)
print(f"Mejores parámetros: {modelo.best_params_}")
print(f"Mejor puntuación: {modelo.best_score_}")

Mejores parámetros: {'criterion': 'entropy', 'max_depth': 11, 'n_estimators': 25}
Mejor puntuación: 0.77473049825991


### <font color='264CC7'> Optimización por RandomSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica RandomSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Usa RandomSearchCV con 25 iteraciones.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [ ]:
parametros_random = {'n_estimators': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
                     'max_depth': [None, 3, 5, 8, 11, 14, 17, 20],
                     'criterion': ['gini', 'entropy', 'log_loss'],
                     'min_samples_split': [2, 5, 10, 15, 20],
                     'min_samples_leaf': [1, 2, 4, 6, 8],
                     'max_features': ['sqrt', 'log2', None]}

modelo_random = RandomizedSearchCV(modelo_base, parametros_random, n_iter=25, cv=k_fold, scoring='accuracy', random_state=68)
modelo_random.fit(X, y)

print(f"Mejores parámetros (RandomSearch): {modelo_random.best_params_}")
print(f"Mejor puntuación (RandomSearch): {modelo_random.best_score_}")

Mejores parámetros (RandomSearch): {'n_estimators': 30, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'criterion': 'entropy'}
Mejor puntuación (RandomSearch): 0.7773194126135302


### <font color='264CC7'> Guardado de modelo </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Con los parámetros óptimos que mejor resultado dieron, reentrena el modelo, muestra su score y guárdalo.
</div>

In [13]:
# Usar el mejor modelo encontrado por RandomizedSearchCV
mejor_modelo = modelo_random.best_estimator_

# Realizar predicciones y evaluar el modelo
y_pred = modelo_random.predict(X)

# Precisión del modelo con dos decimales
accuracy = round(accuracy_score(y, y_pred), 2)
print("Precisión del modelo:", accuracy)

# Matriz de confusión
cm = confusion_matrix(y, y_pred)
print("Matriz de confusión:")
print(cm)

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y, y_pred))

Precisión del modelo: 0.91
Matriz de confusión:
[[476  24]
 [ 47 221]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93       500
           1       0.90      0.82      0.86       268

    accuracy                           0.91       768
   macro avg       0.91      0.89      0.90       768
weighted avg       0.91      0.91      0.91       768



**Guardar el modelo**

In [14]:
# Guardar el modelo
with open('modelo_diabetes.pkl', 'wb') as file:
    pickle.dump(modelo_random, file)

with open('modelo_diabetes.joblib', 'wb') as file:
    joblib.dump(modelo_random, file)

### <font color='264CC7'> Publicación </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Coloca este cuaderno y el modelo en tu repositorio de GitHub. Agrega una licencia MIT y un README.md donde se explique el contenido del repositorio, los datos utilizados y los resultados obtenidos.
</div>